<a href="https://colab.research.google.com/github/SamLutz/2024_MLEES/blob/main/final_project_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final Project MLEES SA24


In [ ]:
!pip install netCDF4
!pip install xarray
import os
import numpy as np
import xarray as xr
from netCDF4 import Dataset
from datetime import datetime
import pooch
import netCDF4 as nc
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.9 MB/s eta 0:00:00


In [ ]:
# Retrieve the files from the cloud using Pooch.
data_url = 'https://unils-my.sharepoint.com/:u:/g/personal/samuel_lutz_unil_ch/Efe70_k3wXJHnrbYK9h1uBwBUgNCzc5_24r73e1xETG0ug?download=1'

hash = None
files = pooch.retrieve(data_url, known_hash=hash)
# Directory containing the NetCDF files
data = nc.Dataset(files)
# Prepare a list to store data from all files
data.variables.keys()


SHA256 hash of downloaded file: ed7070a75e387b8a58b6aaea243f0bb48c9c7722c65c694c11f62bad24c2ebc1
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


dict_keys(['Batt', 'Ptemp', 'AirTC', 'RH', 'Slrw', 'Slrm', 'WS', 'WindDir', 'Rain', 'BP', 'WindGust', 'time_qual', 'Batt_qual', 'Ptemp_qual', 'AirTC_qual', 'RH_qual', 'Slrw_qual', 'Slrm_qual', 'WS_qual', 'WindDir_qual', 'Rain_qual', 'BP_qual', 'WindGust_qual', 'time'])

KeyError: 'surfacetemp'

In [ ]:
# Loop through each NetCDF file in the directory
for filename in os.listdir(data):
    if filename.endswith('.nc'):
        file_path = os.path.join(data, filename)

        # Open the NetCDF file
        with Dataset(file_path, 'r') as nc_file:
            # Extract variables
            unix_time = nc_file.variables['time'][:]
            air_temp = nc_file.variables['AirTC'][:]
            solar_radiation = nc_file.variables['Slrw'][:]
            wind_speed = nc_file.variables['WS'][:]

            # Convert Unix time to datetime
            datetime_times = [datetime.utcfromtimestamp(t) for t in unix_time]

            # Combine into a structured array and append to list
            file_data = np.array(list(zip(datetime_times, air_temp, solar_radiation, wind_speed)),
                                 dtype=[('time', 'O'), ('AirTC', 'f4'), ('SlrW', 'f4'), ('WS', 'f4')])
            data_list.append(file_data)

# Combine all data into a single NumPy array
meteo = np.concatenate(data_list)

# Verify contents
print(meteo[:5])  # Display the first 5 records

In [ ]:
surface_temp=data.variables["surfacetemp"][:]
time=data.variables["time"][:]/86400-18077.916666666668 # bring the time to be 0 on the 1st of january 2020
temp_air=data.variables["temp"][:]